In [2]:
import csv

def append_to_csv(file_path: str, prompt: str, response: str) -> None:
    """
    Append a prompt and response to a CSV file with existing columns 'Prompt' and 'Response'.

    Parameters:
    - file_path: str : Path to the CSV file.
    - prompt: str : The input prompt to be logged.
    - response: str : The response to be logged.
    """
    with open(file_path, mode='a', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['Prompt', 'Response'])
        writer.writerow({'Prompt': prompt, 'Response': response})

# Example usage
file_path = '/home/niel77/MechDesignAgents/Cadquery_output.csv'
prompt = "Create a CAD model of NACA 2412 wing."
code = """
# filename: naca_2412_wing.py
import cadquery as cq
from ocp_vscode import *  # never forget this line
import numpy as np

# Step 1: Define Parameters
chord_length = 1.0  # meters
span = 10.0  # meters
airfoil_thickness = 0.12  # 12% thickness
resolution = 100  # number of points to define the airfoil

# Step 2: Generate NACA 2412 Airfoil Coordinates
def naca_2412_coordinates(chord, thickness, num_points):
    m = 0.02  # maximum camber
    p = 0.4   # location of maximum camber
    t = thickness

    x = np.linspace(0, chord, num_points)
    yt = 5 * t * chord * (0.2969 * np.sqrt(x/chord) - 0.1260 * (x/chord) - 0.3516 * (x/chord)**2 + 0.2843 * (x/chord)**3 - 0.1015 * (x/chord)**4)
    yc = np.where(x < p * chord, 
                  m * (x / (p**2)) * (2 * p - x / chord), 
                  m * ((chord - x) / ((1 - p)**2)) * (1 + x / chord - 2 * p))

    xu = x - yt * np.sin(np.arctan(np.gradient(yc, x)))
    xl = x + yt * np.sin(np.arctan(np.gradient(yc, x)))
    yu = yc + yt * np.cos(np.arctan(np.gradient(yc, x)))
    yl = yc - yt * np.cos(np.arctan(np.gradient(yc, x)))

    upper_surface = np.column_stack((xu, yu))
    lower_surface = np.column_stack((xl, yl))

    return np.vstack((upper_surface, lower_surface[::-1]))

coordinates = naca_2412_coordinates(chord_length, airfoil_thickness, resolution)

# Step 3: Create the CAD Model
airfoil_wire = cq.Workplane("XY").polyline(coordinates.tolist()).close()
wing = airfoil_wire.extrude(span)

# Step 4: Save the Model
cq.exporters.export(wing, "naca_2412_wing.stl")
cq.exporters.export(wing, "naca_2412_wing.step")

# Step 5: Visualize the Model
show_object(wing)  # always visualize the model
"""

response = f"```python\n+{code}\n```"
append_to_csv(file_path, prompt, response)